# Kernel Semantic cu Integrarea Serverului OpenBnB MCP

Acest notebook demonstrează cum să utilizezi Kernel Semantic cu serverul real OpenBnB MCP pentru a căuta cazare Airbnb folosind MCPStdioPlugin. Pentru accesul la LLM, se folosește Azure AI Foundry. Pentru configurarea variabilelor de mediu, poți urma [Lecția de Configurare](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Crearea conexiunii pluginului MCP

Ne vom conecta la [serverul MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) folosind MCPStdioPlugin. Acest server oferă funcționalitate de căutare Airbnb prin pachetul @openbnb/mcp-server-airbnb.


## Crearea Clientului

În acest exemplu, vom utiliza Azure AI Foundry pentru accesul nostru la LLM. Asigură-te că variabilele de mediu sunt configurate corect.


## Configurarea Mediului

Configurează setările Azure OpenAI. Asigură-te că ai setat următoarele variabile de mediu:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Înțelegerea integrării OpenBnB MCP

Acest notebook se conectează la **serverul real OpenBnB MCP**, care oferă funcționalitatea de căutare Airbnb.

### Cum funcționează:

1. **MCPStdioPlugin**: Utilizează comunicarea prin intrare/ieșire standard cu serverul MCP
2. **Pachet NPM real**: Descarcă și rulează `@openbnb/mcp-server-airbnb` prin npx
3. **Date în timp real**: Returnează date reale despre proprietăți Airbnb din API-urile lor
4. **Descoperirea funcțiilor**: Agentul descoperă automat funcțiile disponibile de la serverul MCP

### Funcții disponibile:

Serverul OpenBnB MCP oferă, de obicei:
- **search_listings** - Caută proprietăți Airbnb după locație și criterii
- **get_listing_details** - Obține informații detaliate despre proprietăți specifice
- **check_availability** - Verifică disponibilitatea pentru date specifice
- **get_reviews** - Recuperează recenzii pentru proprietăți
- **get_host_info** - Obține informații despre gazdele proprietăților

### Cerințe preliminare:

- **Node.js** instalat pe sistemul tău
- **Conexiune la internet** pentru a descărca pachetul serverului MCP
- **NPX** disponibil (vine împreună cu Node.js)

### Testarea conexiunii:

Poți testa manual serverul MCP rulând:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Aceasta va descărca și porni serverul OpenBnB MCP, la care Semantic Kernel se conectează ulterior pentru a obține date reale de la Airbnb.


## Rularea Agentului cu Serverul OpenBnB MCP

Acum vom rula Agentul AI care se conectează la serverul OpenBnB MCP pentru a căuta cazare reală pe Airbnb în Stockholm pentru 2 adulți și 1 copil. Simțiți-vă liber să modificați lista `user_inputs` pentru a schimba criteriile de căutare.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Rezumat
Felicitări! Ai reușit să construiești un agent AI care se integrează cu căutarea reală de cazare utilizând Model Context Protocol (MCP):

## Tehnologii utilizate:
- Semantic Kernel - Pentru construirea agenților inteligenți cu Azure OpenAI
- Azure AI Foundry - Pentru capabilități LLM și completarea conversațiilor
- MCP (Model Context Protocol) - Pentru integrarea standardizată a instrumentelor
- OpenBnB MCP Server - Pentru funcționalitatea reală de căutare Airbnb
- Node.js/NPX - Pentru rularea serverului MCP extern

## Ce ai învățat:
- Integrarea MCP: Conectarea agenților Semantic Kernel la servere MCP externe
- Acces la date în timp real: Căutarea proprietăților Airbnb reale prin API-uri live
- Comunicare prin protocol: Utilizarea comunicării stdio între agent și serverul MCP
- Descoperirea funcțiilor: Identificarea automată a funcțiilor disponibile de la serverele MCP
- Răspunsuri în flux: Capturarea și înregistrarea apelurilor de funcții în timp real
- Randare HTML: Formatarea răspunsurilor agentului cu tabele stilizate și afișaje interactive

## Pașii următori:
- Integrarea altor servere MCP (vreme, zboruri, restaurante)
- Construirea unui sistem multi-agent care combină protocoalele MCP și A2A
- Crearea de servere MCP personalizate pentru propriile surse de date
- Implementarea memoriei conversaționale persistente între sesiuni
- Implementarea agentului în Azure Functions cu orchestrarea serverului MCP
- Adăugarea autentificării utilizatorilor și a capabilităților de rezervare



---

**Declinarea responsabilității**:  
Acest document a fost tradus utilizând serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru a asigura acuratețea, vă rugăm să aveți în vedere că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa nativă ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
